In [1]:
import pandas as pd
import numpy as np

#col_range = [i for i in range(0,28)]
# By default the "date" column was in string format, 
#  we need to convert it into date-time format 
# parse_dates =["date"], converts the "date" column to date-time format 
  
# Resampling works with time-series data only 
# so convert "date" column to index
df = pd.read_csv("/content/sample_data/covid19_cases_switzerland_openzh-phase2.csv", usecols= np.r_[0:28], parse_dates =["Date"]) # First 27 columns

df.tail()

,Date,AG,AI,AR,BE,BL,BS,FR,GE,GL,GR,JU,LU,NE,NW,OW,SG,SH,SO,SZ,TG,TI,UR,VD,VS,ZG,ZH,CH
143,2020-10-21,3250.0,154.0,304.0,5043.0,899.0,751.0,3583.0,7492.0,181.0,775.0,833.0,1754.0,1821.0,179.0,184.0,3509.0,259.0,955.0,1538.0,1185.0,1594.0,169.0,11065.0,4835.0,873.0,10441.0,63626.0
144,2020-10-22,3516.0,172.0,330.0,5595.0,965.0,826.0,3892.0,8184.0,187.0,867.0,878.0,1935.0,1835.0,195.0,193.0,3629.0,290.0,1056.0,1654.0,1237.0,1717.0,NaN,11857.0,4868.0,947.0,11222.0,68216.0
145,2020-10-23,NaN,187.0,382.0,6397.0,1063.0,900.0,4390.0,NaN,201.0,952.0,962.0,2119.0,NaN,211.0,200.0,NaN,317.0,1156.0,1759.0,1237.0,2012.0,189.0,12601.0,NaN,1010.0,11877.0,72154.0
146,2020-10-24,NaN,197.0,NaN,7112.0,1182.0,1003.0,4885.0,NaN,NaN,1042.0,NaN,NaN,NaN,233.0,NaN,NaN,NaN,NaN,1858.0,NaN,2269.0,NaN,NaN,NaN,NaN,NaN,74064.0
147,2020-10-25,NaN,NaN,NaN,7646.0,NaN,1084.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2525.0,NaN,NaN,NaN,NaN,NaN,74935.0


In [2]:
df.shape # setting date as index causes decrement in column clount 28 to 27

(148, 28)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148 entries, 0 to 147
Data columns (total 28 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    148 non-null    datetime64[ns]
 1   AG      145 non-null    float64       
 2   AI      82 non-null     float64       
 3   AR      69 non-null     float64       
 4   BE      134 non-null    float64       
 5   BL      147 non-null    float64       
 6   BS      148 non-null    float64       
 7   FR      147 non-null    float64       
 8   GE      145 non-null    float64       
 9   GL      104 non-null    float64       
 10  GR      147 non-null    float64       
 11  JU      146 non-null    float64       
 12  LU      115 non-null    float64       
 13  NE      145 non-null    float64       
 14  NW      147 non-null    float64       
 15  OW      101 non-null    float64       
 16  SG      145 non-null    float64       
 17  SH      104 non-null    float64       
 18  SO      99

In [4]:
df.set_index(['Date'],inplace=True) # set index
df[0:1] = 0 #initialize first row with zero

In [5]:
df.shape

(148, 27)

In [6]:
df.drop(columns=['CH'],inplace=True, axis=1)
df.shape

(148, 26)

In [7]:
df.head()

,AG,AI,AR,BE,BL,BS,FR,GE,GL,GR,JU,LU,NE,NW,OW,SG,SH,SO,SZ,TG,TI,UR,VD,VS,ZG,ZH
Date,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-05-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-06-01,0.0,NaN,NaN,0.0,1.0,0.0,0.0,1.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2020-06-02,0.0,NaN,0.0,1.0,1.0,0.0,2.0,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,4.0,2.0,0.0,2.0
2020-06-03,1.0,0.0,0.0,2.0,2.0,1.0,3.0,6.0,0.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,NaN,6.0,2.0,0.0,5.0
2020-06-04,3.0,NaN,0.0,4.0,2.0,1.0,5.0,8.0,0.0,1.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,1.0,NaN,9.0,4.0,0.0,13.0


In [8]:
np.isnan(df).sum()
df.shape

(148, 26)

In [9]:
def fill_missing(df):
    for row in range(df.shape[0]):
        for col in range(df.shape[1]):
            if np.isnan(df[row, col]):
                df[row, col] = df[row-1, col]        

fill_missing(df.values)

np.isnan(df).sum()


AG    0
AI    0
AR    0
BE    0
BL    0
BS    0
FR    0
GE    0
GL    0
GR    0
JU    0
LU    0
NE    0
NW    0
OW    0
SG    0
SH    0
SO    0
SZ    0
TG    0
TI    0
UR    0
VD    0
VS    0
ZG    0
ZH    0
dtype: int64

In [10]:
df.to_csv('/content/sample_data/exported.csv', index = True)

In [11]:
daily_changes_df = pd.DataFrame()

def calculate_daily_changes(data_toBe_treated):
  for row in range(data_toBe_treated.shape[0]):
    for col in range(data_toBe_treated.shape[1]):
      daily_changes_df.loc[row, col] = data_toBe_treated[row, col] - data_toBe_treated[row-1, col]


calculate_daily_changes(df.values)

daily_changes_df.columns = ['AG', 'AI', 'AR', 'BE', 'BL', 'BS', 'FR', 'GE', 'GL', 'GR', 'JU', 'LU',
        'NE', 'NW', 'OW', 'SG', 'SH', 'SO', 'SZ', 'TG', 'TI', 'UR', 'VD', 'VS',
        'ZG', 'ZH']

# daily_changes_df.set_index(df.index)

daily_changes_df.index=df.index

daily_changes_df.fillna(0)
daily_changes_df[0:1] = 0.0

In [12]:
daily_changes_df.to_csv('/content/sample_data/changes_daily.csv', index = True)

In [13]:

dnc_avg = daily_changes_df.rolling(3).mean()
dnc_avg.fillna(0)

# nan to zero
dnc_avg = dnc_avg.replace(np.nan, 0)

#rouding and converting to int
dnc_avg = dnc_avg.round().astype(int)

dnc_avg.to_csv('/content/sample_data/average_cases.csv', index = True)

dnc_avg

,AG,AI,AR,BE,BL,BS,FR,GE,GL,GR,JU,LU,NE,NW,OW,SG,SH,SO,SZ,TG,TI,UR,VD,VS,ZG,ZH
Date,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-05-31,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2020-06-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2020-06-02,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1
2020-06-03,0,0,0,1,1,0,1,2,0,0,0,0,1,0,0,0,0,2,0,0,0,0,2,1,0,2
2020-06-04,1,0,0,1,0,0,2,2,0,0,0,0,2,0,0,0,0,2,0,0,0,0,3,1,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-10-21,205,21,33,321,64,31,243,462,20,39,64,199,157,12,15,347,20,92,143,103,139,21,683,478,77,506
2020-10-22,264,16,29,437,72,46,277,573,12,63,67,170,97,14,8,258,19,73,122,108,172,10,755,302,62,698
2020-10-23,179,19,35,622,79,64,365,354,13,79,70,177,43,16,10,144,23,88,127,56,224,15,773,132,66,808


In [14]:
#need if required or not

Cantons = dnc_avg.to_dict()
Cantons

{'AG': {Timestamp('2020-05-31 00:00:00'): 0,
  Timestamp('2020-06-01 00:00:00'): 0,
  Timestamp('2020-06-02 00:00:00'): 0,
  Timestamp('2020-06-03 00:00:00'): 0,
  Timestamp('2020-06-04 00:00:00'): 1,
  Timestamp('2020-06-05 00:00:00'): 1,
  Timestamp('2020-06-06 00:00:00'): 1,
  Timestamp('2020-06-07 00:00:00'): 1,
  Timestamp('2020-06-08 00:00:00'): 1,
  Timestamp('2020-06-09 00:00:00'): 1,
  Timestamp('2020-06-10 00:00:00'): 1,
  Timestamp('2020-06-11 00:00:00'): 2,
  Timestamp('2020-06-12 00:00:00'): 2,
  Timestamp('2020-06-13 00:00:00'): 2,
  Timestamp('2020-06-14 00:00:00'): 2,
  Timestamp('2020-06-15 00:00:00'): 1,
  Timestamp('2020-06-16 00:00:00'): 1,
  Timestamp('2020-06-17 00:00:00'): 1,
  Timestamp('2020-06-18 00:00:00'): 1,
  Timestamp('2020-06-19 00:00:00'): 2,
  Timestamp('2020-06-20 00:00:00'): 3,
  Timestamp('2020-06-21 00:00:00'): 2,
  Timestamp('2020-06-22 00:00:00'): 2,
  Timestamp('2020-06-23 00:00:00'): 1,
  Timestamp('2020-06-24 00:00:00'): 3,
  Timestamp('2020-0

In [15]:
cantons = df.columns
date = df.index.strftime('%m/%Y')

cantons, date

(Index(['AG', 'AI', 'AR', 'BE', 'BL', 'BS', 'FR', 'GE', 'GL', 'GR', 'JU', 'LU',
        'NE', 'NW', 'OW', 'SG', 'SH', 'SO', 'SZ', 'TG', 'TI', 'UR', 'VD', 'VS',
        'ZG', 'ZH'],
       dtype='object'),
 Index(['05/2020', '06/2020', '06/2020', '06/2020', '06/2020', '06/2020',
        '06/2020', '06/2020', '06/2020', '06/2020',
        ...
        '10/2020', '10/2020', '10/2020', '10/2020', '10/2020', '10/2020',
        '10/2020', '10/2020', '10/2020', '10/2020'],
       dtype='object', name='Date', length=148))

In [20]:
from bokeh.plotting import figure, output_file, show, output_notebook, save
from bokeh.models import ColumnDataSource, Line, HoverTool
from bokeh.palettes import Spectral6
from bokeh.transform import factor_cmap
from bokeh.palettes import Dark2

output_notebook()


# # Create a color list to represent different cantons in the plot, you can either construct your own color patette or use the Bokeh color pallete
# color_palette = factor_cmap('Date', palette=Spectral6, factors=cantons)

# # Build a dictionary with date and each canton name as a key, i.e., {'date':[], 'AG':[], ..., 'ZH':[]}
# # For each canton, the value is a list containing the averaged daily new cases
source_dict= {'Date':date.values, 'AG':dnc_avg['AG'], 'AI':dnc_avg.AI.values, 'AR':dnc_avg.AR.values, 'BE':dnc_avg.BE.values, 'BL':dnc_avg.BL.values, 
              'BS':dnc_avg.BS.values, 'FR':dnc_avg.FR.values,
              'GE':dnc_avg.GE.values, 'GL':dnc_avg.GL.values, 'GR':dnc_avg.GR.values, 'JU':dnc_avg.JU.values, 'LU':dnc_avg.LU.values,
        'NE':dnc_avg.NE.values, 'NW':dnc_avg.NW.values, 'OW':dnc_avg.OW.values, 'SG':dnc_avg.SG.values, 'SH':dnc_avg.SH.values, 'SO':dnc_avg.SO.values, 
        'SZ':dnc_avg.SZ.values, 'TG':dnc_avg.TG.values, 'TI':dnc_avg.TI.values, 'UR':dnc_avg.UR.values, 'VD':dnc_avg.VD.values, 'VS':dnc_avg.VS.values,
        'ZG':dnc_avg.ZG.values, 'ZH':dnc_avg.ZH.values}


source= ColumnDataSource(dnc_avg)

p = figure(plot_width=1000, plot_height=800, x_axis_type="datetime", toolbar_location=None)
p.line(x='Date', y='AG', line_width=1, source=source, legend_label='AG',name='AG')
p.line(x='Date', y='AI', line_width=1, source=source,color = '#31446B', legend_label='AI',name='AI')
p.line(x='Date', y='AR', line_width=1, source=source,color = '#084594', legend_label='AR',name='AR')
p.line(x='Date', y='BE', line_width=1, source=source,color = '#ff7f0e', legend_label='BE',name='BE')
p.line(x='Date', y='BL', line_width=1, source=source,color = '#aec7e8', legend_label='BL',name='BL')
p.line(x='Date', y='BS', line_width=1, source=source,color = '#2ca02c', legend_label='BS',name='BS')
p.line(x='Date', y='FR', line_width=1, source=source,color = '#d62728', legend_label='FR',name='FR')
p.line(x='Date', y='GE', line_width=1, source=source,color = '#9467bd', legend_label='GE',name='GE')
p.line(x='Date', y='GL', line_width=1, source=source,color = '#8c564b', legend_label='GL',name='GL')
p.line(x='Date', y='GR', line_width=1, source=source,color = '#e377c2', legend_label='GR',name='GR')
p.line(x='Date', y='JU', line_width=1, source=source,color = '#7f7f7f', legend_label='JU',name='JU')
p.line(x='Date', y='LU', line_width=1, source=source,color = '#bcbd22', legend_label='LU',name='LU')
p.line(x='Date', y='NE', line_width=1, source=source,color = '#17becf', legend_label='NE',name='NE')
p.line(x='Date', y='NW', line_width=1, source=source,color = '#9c9ede', legend_label='NW',name='NW')
p.line(x='Date', y='OW', line_width=1, source=source,color = '#637939', legend_label='OW',name='OW')
p.line(x='Date', y='SG', line_width=1, source=source,color = '#cedb9c', legend_label='SG',name='SG')
p.line(x='Date', y='SH', line_width=1, source=source,color = '#8c6d31', legend_label='SH',name='SH')
p.line(x='Date', y='SO', line_width=1, source=source,color = '#bd9e39', legend_label='SO',name='SO')
p.line(x='Date', y='SZ', line_width=1, source=source,color = '#e7ba52', legend_label='SZ',name='SZ')
p.line(x='Date', y='TG', line_width=1, source=source,color = '#843c39', legend_label='TG',name='TG')
p.line(x='Date', y='TI', line_width=1, source=source,color = '#de9ed6', legend_label='TI',name='TI')
p.line(x='Date', y='UR', line_width=1, source=source,color = '#d62728', legend_label='UR',name='UR')
p.line(x='Date', y='VD', line_width=1, source=source,color = '#e377c2', legend_label='VD',name='VD')
p.line(x='Date', y='VS', line_width=1, source=source,color = '#17becf', legend_label='VS',name='VS')
p.line(x='Date', y='ZG', line_width=1, source=source,color = '#7f7f7f', legend_label='ZG',name='ZG')
p.line(x='Date', y='ZH', line_width=1, source=source,color = '#bdbdbd', legend_label='ZH',name='ZH')
p.title.text = 'Daily New Cases in Switzerland'
p.legend.location = "top_left"
p.legend.click_policy="hide"
hover = p.select(dict(type=HoverTool))
hover = HoverTool(tooltips=[('Date', '@Date{%F}'),('Cantons','$name'),("Cases", "$y{0}")], formatters={'@Date': 'datetime'})
p.add_tools(hover)
mode='vline'

show(p)

# # ### Task 2: Data Visualization

# # ## T2.1: Draw a group of lines, each line represents a canton, using date, dnc_avg as x,y. Add proper legend.
# # # https://docs.bokeh.org/en/latest/docs/reference/models/glyphs/line.html?highlight=line#bokeh.models.glyphs.Line
# # # https://docs.bokeh.org/en/latest/docs/user_guide/interaction/legends.html

# p = figure(plot_width=1000, plot_height=800, x_axis_type="datetime")
# p.title.text = 'Daily New Cases in Switzerland'

# # lines = []
# # for canton in zip(cantons): 
# #   lines.append(Line(x="x", y="y", line_width=6, line_alpha=0.6, legend=canton)) #line_color=color,
# #   p.line(x="xx", y="yy", line_width=6, line_alpha=0.6, legend=canton, line_color={'field': 'yy', 'transform': color_mapper}, source=source)

# # ts_list_of_list = []
# # for i in range(0,len(toy_df.columns)):
# #     ts_list_of_list.append(toy_df.index)

# # vals_list_of_list = toy_df.values.T.tolist()

# p.multi_line(xs='Date', ys='AR', color='red', line_width=2, source=source)

# # # Make the legend of the plot clickable, and set the click_policy to be "hide"
# # ## T2.2 Add hovering tooltips to display date, canton and averaged daily new case

# # # (Hovertip doc) https://docs.bokeh.org/en/latest/docs/user_guide/tools.html#hovertool
# # # (Date hover)https://stackoverflow.com/questions/41380824/python-bokeh-hover-date-time
# # hover = ...

# # p.add_tools(hover)

# show(p)
# output_file("dvc_ex9.html")
# save(p)
